# Warm up 14 - Vehicle Stock in China

TEP4290, Spring 2024

## Preparation: Packages and data imports

Start by importing the necessary packages:

In [23]:
# Write all the necessary imports here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from dynamic_stock_model import DynamicStockModel

Then import the data into a Dataframe using the pandas read_excel function (check the documentation to find out which arguments to use) and clean it.

In [24]:
#Working directory ?
working_directory = os.getcwd()
print(working_directory)

c:\Users\jujug\Documents\Ecole\4. NTNU\MFA2\TEP4290\TEP4290-juliette\Warm_up_14_chinese_vehicle_fleet


In [25]:
# Extracting data from the source file into a panda Dataframe
file = "Warm_up_14_data.xls"
# This file should be placed in the same directory as this notebook (on the JupyterHub server)

# We read the data and store it in a Dataframe called "data", using the read_excel() panda function
data = pd.read_excel(file, sheet_name="CarStockData",usecols="D:P", header =1,)

# Drop the first two rows
data = data.drop(index=[0,1])
data = data.reset_index(drop=True)

# This is just to replace all the void values in "data" by 0, which makes things easier
data.fillna(0, inplace=True)


C:\Users\jujug\AppData\Local\Temp\ipykernel_14812\3479403728.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)


We create two Dataframes from data:
 - historic_data with values up to 2008
 - future_data with values for 2009 and after


In [26]:
historic_data = data[data.Time < 2009]
future_data = data[data.Time >= 2009]

## Inflow-driven model

Calculate outflows, stock change and stock for historic data, using an inflow-driven model:

In [27]:
# Parameters of the lifetime distribution
mean_lifetime = 15
stddev_lifetime = 5
n_years = len(historic_data["Time"])



lt_mean = np.full(n_years, mean_lifetime)
lt_std = np.full(n_years, stddev_lifetime)

# Reset the index 
historic_data = historic_data.reset_index(drop=True)

# Maintenant crée ton modèle
inflow_driven_model = DynamicStockModel(
    t = historic_data["Time"].values, # Utilise .values pour passer des tableaux propres
    i = historic_data["Inflow"].values, 
    lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std}
)
# Create a new attribute in the class dynamic stock model
inflow_driven_model = DynamicStockModel(
    t = historic_data["Time"], 
    i = historic_data["Inflow"], 
    lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std })

# Compute the entire model
Stock_by_cohort, ExitFlag = inflow_driven_model.compute_s_c_inflow_driven()



In [28]:
# Add the outflow in the historic_data

inflow_driven_model.o_c = None
Outflow_by_cohort, ExitFlag = inflow_driven_model.compute_o_c_from_s_c()
outflow_total = Outflow_by_cohort.sum(axis=1)
historic_data["Outflow"] = outflow_total

# Compute the stock change in historic data
historic_data["Stock Change"] = historic_data["Inflow"] - historic_data["Outflow"]

#Compute stock in historic data
historic_data["Stock"] = historic_data["Stock Change"].cumsum()


In [29]:
# merge historic data and data

data["Outflow"] = historic_data["Outflow"]
data["Stock Change"] = historic_data["Stock Change"]
data["Stock"] = historic_data["Stock"]

## Stock-driven model

Calculate stock change, outflows and inflows for future data, using a stock-driven model.
You need to run the stock-driven model for all years, combining historic stock data from the inflow-driven model, and future stock data from population and car ownership projections.

In [35]:
#Compute the stock between 2009 and 2050
mask = data['Time'] >= 2009
data.loc[mask, "Stock"] = data.loc[mask, "cars per 1000 people"] * data.loc[mask, "Population"]

In [36]:
# Parameters of the lifetime distribution for the entire data
n_years = len(data["Time"])
lt_mean = np.full(n_years, mean_lifetime)
lt_std = np.full(n_years, stddev_lifetime)

# Create the stock driven model
stock_driven_model = DynamicStockModel(t = data["Time"], 
                                       s = data["Stock"], 
                                       lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std })

CheckStr, ExitFlag = stock_driven_model.dimension_check()
print(CheckStr)


<br><b> Checking dimensions of dynamic stock model DSM.Time vector is present with 101 years.<br>Inflow is not present.<br>Total stock is present with 101 years.<br>Stock by cohorts is not present.<br>Total outflow is not present.<br>Outflow by cohorts is not present.<br>Lifetime distribution is present with type Normal and mean [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15].<br>


In [37]:
# Compute the entire model

S_C, O_C, I, ExitFlag = stock_driven_model.compute_stock_driven_model()
O, ExitFlag   = stock_driven_model.compute_outflow_total()
DS, ExitFlag  = stock_driven_model.compute_stock_change()
Bal, ExitFlag = stock_driven_model.check_stock_balance()
print(Bal)
     

[ 0.00000000e+00  0.00000000e+00  4.54747351e-13  0.00000000e+00
  4.54747351e-13  0.00000000e+00  9.09494702e-13 -1.36424205e-12
 -9.09494702e-13  9.09494702e-13 -3.18323146e-12 -4.54747351e-13
  3.63797881e-12  1.81898940e-12  0.00000000e+00 -4.54747351e-12
  2.72848411e-12  4.54747351e-12  9.09494702e-13 -2.27373675e-12
 -1.81898940e-12  1.81898940e-12 -7.27595761e-12  0.00000000e+00
  1.81898940e-11  2.18278728e-11 -2.91038305e-11  3.63797881e-12
  0.00000000e+00 -7.27595761e-12  1.45519152e-11 -4.36557457e-11
  5.09317033e-11  7.27595761e-12  1.45519152e-11  0.00000000e+00
 -8.73114914e-11 -5.82076609e-11  1.45519152e-10  8.73114914e-11
 -1.74622983e-10  1.16415322e-10 -1.16415322e-10  2.32830644e-10
 -3.49245965e-10 -1.16415322e-10  4.65661287e-10 -5.82076609e-10
 -3.49245965e-10  8.14907253e-10  0.00000000e+00  4.65661287e-10
  6.98491931e-10 -1.39698386e-09  9.31322575e-10 -1.86264515e-09
 -9.31322575e-10  1.86264515e-09 -1.86264515e-09  1.86264515e-09
 -8.38190317e-09  5.58793

In [38]:
# extract the outflow 
stock_driven_model.o_c = None 
Outflow_by_cohort, ExitFlag_O = stock_driven_model.compute_o_c_from_s_c()
outflow_total = Outflow_by_cohort.sum(axis=1)

# add the outflow and inflow in future data
data["Outflow"] = outflow_total
data["Inflow"] = I

# compute stock and stock change in future data
data["Stock Change"] = data["Inflow"] - data["Outflow"]
data["Stock"] = S_C.sum(axis=1)

In [39]:
# Save the data to a new excel file
data.to_excel("Warm_up_14_new_data.xlsx", index=False)

## Final calculations

Perform the calculations for distance driven, fuel consumption and CO2 emissions

In [ ]:
# distance driven 



## Plot the results

Plot the results of the model for inflows and outflows, stock change, stock, total distance driven, total fuel consumption, and total CO2 emissions

## Exports the results

Export the results to a new Excel file - you can then in the end before submission copy the stock over to the existing data if you prefer to, but it is not required of you

## Interpretation